In [1]:
x=1

In [2]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import decoupler as dc
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox
import pertpy 

import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

To use sccoda or tasccoda please install ete3 with pip install ete3

In [2]:
import scanpy as sc

In [3]:
adata = sc.read('../../../pipeline/data/multigrate_mil/hlca_healthy_copd_embed_balanced/1fe5654238_adata_both.h5ad')
adata

AnnData object with n_obs × n_vars = 152766 × 30
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_leve

In [4]:
hlca = sc.read('../../../../snakemake/hlca/5fold/hlca_hyperparam/data/pp/hlca_hvg_pearson.h5ad')
adata.obsm['counts'] = hlca[adata.obs_names].X

In [9]:
adata.write('../../../../data_to_share/hlca_healthy_ipf_subset.h5ad')

In [5]:
adata.obsm['counts'].data

array([ 1.,  1.,  2., ..., 15.,  1.,  6.], dtype=float32)

In [6]:
adata.obsm['counts']

<152766x2000 sparse matrix of type '<class 'numpy.float32'>'
	with 46181614 stored elements in Compressed Sparse Row format>

In [7]:
import rpy2.robjects as ro

def run_de(adata):
    r=ro.r
    r.source("test.R")
    df = r.fit_model(adata)
    return df

In [48]:
for ct in np.unique(adata.obs['transf_ann_level_3_label'].astype(str)):
    print(f'Processing {ct}...')
    adata_ct = adata[adata.obs['transf_ann_level_3_label'] == ct].copy()
    adata_ct_disease = adata_ct[adata_ct.obs['lung_condition_coarse'] == 'COPD'].copy()

    print(len(adata_ct_disease))
    
    threshold_idx = int(len(adata_ct_disease) * 0.9)
    threshold_value = sorted(adata_ct_disease.obs['cell_attn'])[threshold_idx]
    
    adata_ct_disease.obs['group'] = 'all'
    adata_ct_disease.obs.loc[adata_ct_disease.obs['cell_attn'] >= threshold_value, 'group'] = 'top'

    # adata_ct_disease.obs['sample_group'] = adata_ct_disease.obs['sample'].astype('str') + '_' + adata_ct_disease.obs['group']
    # adata_ct_disease.obs['sample_group'] = adata_ct_disease.obs['sample_group'].astype('category')

    # samples_to_exclude = []
    # for sample in np.unique(adata_ct_disease.obs[['sample_group']]):
    #     sample_len = len(adata_ct_disease[adata_ct_disease.obs['sample_group'] == sample])
    #     if sample_len < 10:
    #         samples_to_exclude.append(sample)
    
    # adata_ct_disease = adata_ct_disease[~adata_ct_disease.obs['sample_group'].isin(samples_to_exclude)].copy()

    #adata_ct_disease.X = adata_ct_disease.obsm['counts'].copy()
    adata_pb = sc.AnnData(adata_ct_disease.obsm['counts'])
    adata_pb.obs = adata_ct_disease.obs
    adata_pb.var_names = hlca.var_names

    adata_ = dc.get_pseudobulk(adata_pb, 'sample', 'group', mode='sum', min_cells=-1, min_counts=10)

    adata_r = sc.AnnData(adata_.X, obs=adata_.obs[['group', 'study', 'sample']])
    adata_r.var_names = adata_.var_names
    adata_r.obs['sample'] = adata_r.obs['sample'].str.replace('-', '_')
    adata_r.obs.index = adata_r.obs.index.str.replace('-', '_')

    adata_r = adata_r.copy()

    sc.pp.filter_genes(adata_r, min_cells=10)
    print(adata_r.X.shape)
    if ((adata_r.X.shape[0] == 0) or (adata_r.X.shape[1] == 0)):
        continue
    df = run_de(adata_r)

    df = ro.conversion.rpy2py(df)
    df.to_csv(f'{ct}.csv')

Processing AT1...
(39, 802)
[1] "Dimensions before subsetting:"
[1] 802  39
[1] ""
[1] "Dimensions after subsetting:"
[1] 176  39
[1] ""
Processing AT2...
(43, 1115)
[1] "Dimensions before subsetting:"
[1] 1115   43
[1] ""
[1] "Dimensions after subsetting:"
[1] 230  43
[1] ""
Processing B cell lineage...
(44, 1253)
[1] "Dimensions before subsetting:"
[1] 1253   44
[1] ""
[1] "Dimensions after subsetting:"
[1] 306  44
[1] ""
Processing Basal...
(35, 700)
[1] "Dimensions before subsetting:"
[1] 700  35
[1] ""
[1] "Dimensions after subsetting:"
[1] 110  35
[1] ""
Processing Dendritic cells...
(46, 1427)
[1] "Dimensions before subsetting:"
[1] 1427   46
[1] ""
[1] "Dimensions after subsetting:"
[1] 481  46
[1] ""
Processing EC arterial...
(24, 549)
[1] "Dimensions before subsetting:"
[1] 549  24
[1] ""
[1] "Dimensions after subsetting:"
[1] 113  24
[1] ""
Processing EC capillary...
(34, 772)
[1] "Dimensions before subsetting:"
[1] 772  34
[1] ""
[1] "Dimensions after subsetting:"
[1] 60 34

IndexError: list index out of range